Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [46]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)


In [48]:
def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    # return the wrangled dataframe
    return X

In [49]:
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [50]:
target = 'status_group'

train_features = train.drop(columns=[target])

numeric_features = train_features.select_dtypes(include='number').columns.tolist()

cardinality = train_features.select_dtypes(exclude='number').nunique()

categorical_features = cardinality[cardinality <= 50].index.tolist()

features = numeric_features + categorical_features

In [51]:
x_train = train[features]
y_train = train[target]
x_val = val[features]
y_val = val[target]
x_test = test[features]

In [63]:
import category_encoders as ce
import numpy as np
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler



pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    StandardScaler(), 
    SelectKBest(f_classif, k=20), 
    RandomForestClassifier(n_jobs=-1, random_state=1)
)

k = 3
scores = cross_val_score(pipeline, x_train, y_train, cv=k)

In [0]:
print(f'MAE for {k} folds:', -scores)

In [74]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
    StandardScaler(), 
    SelectKBest(f_classif), 
    RandomForestClassifier()
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'selectkbest__k': range(1, len(x_train.columns)+1), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=5, 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(x_train, y_train);

C:\Users\Matt\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 76 is smaller than n_iter=100. Running 76 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Fitting 5 folds for each of 76 candidates, totalling 380 fits
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   42.7s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 

In [75]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'simpleimputer__strategy': 'mean', 'selectkbest__k': 37}
Cross-validation MAE -0.7977483164983166


In [67]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,74,75,73,72,70,71,68,66,67,69,...,8,7,6,13,5,4,3,2,1,0
mean_fit_time,19.5138,18.3616,20.4151,20.0603,18.6026,19.6108,17.9182,18.6676,19.02,18.9139,...,11.3556,13.2141,11.8886,12.3264,12.8884,11.704,12.6447,11.7102,12.9379,12.527
std_fit_time,0.507671,1.81252,0.592076,0.320672,0.182199,0.255776,0.352903,0.177678,0.373476,0.20939,...,0.198063,0.289719,0.360701,0.214923,0.181367,0.205111,0.247083,0.146178,0.336348,0.165331
mean_score_time,1.63503,1.08645,1.58638,1.51717,1.50813,1.50412,1.54279,1.57406,1.4786,1.51608,...,1.18264,1.37596,1.22134,1.15432,1.28896,1.27526,1.39683,1.21527,1.22314,1.20189
std_score_time,0.210274,0.122132,0.0859009,0.0662348,0.079243,0.0831046,0.065289,0.0737521,0.0803529,0.0301197,...,0.0284862,0.114673,0.0526808,0.0865089,0.100439,0.130678,0.0852344,0.0312986,0.0473277,0.0524388
param_simpleimputer__strategy,mean,median,median,mean,mean,median,mean,mean,median,median,...,mean,median,mean,median,median,mean,median,mean,median,mean
param_selectkbest__k,38,38,37,37,36,36,35,34,34,35,...,5,4,4,7,3,3,2,2,1,1
params,"{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'median', 'selectk...",...,"{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'mean', 'selectkbe..."
split0_test_score,0.753788,0.752525,0.753998,0.753998,0.753577,0.751789,0.754945,0.754524,0.75484,0.75242,...,0.700231,0.700231,0.700231,0.700231,0.698548,0.698548,0.698548,0.698548,0.644781,0.644781
split1_test_score,0.763363,0.763468,0.762626,0.760838,0.761258,0.761785,0.75968,0.760943,0.761469,0.760311,...,0.691393,0.691393,0.691393,0.691393,0.692024,0.692024,0.692024,0.692024,0.637731,0.637731
split2_test_score,0.763152,0.762416,0.76189,0.761574,0.75968,0.761574,0.758312,0.760206,0.759364,0.759049,...,0.698758,0.698758,0.698758,0.698758,0.695812,0.695812,0.695812,0.695812,0.640993,0.640993


In [77]:
pipeline = search.best_estimator_

In [79]:
y_pred = pipeline.predict(x_test)
submission = sample_submission.copy()
submission['status_group'] = y_pred
filename = 'kaggle-waterpumps-submission-03.csv'
submission.to_csv(filename, index=False)

In [78]:
print('Validation Accuracy', pipeline.score(x_val, y_val))

Validation Accuracy 0.8113636363636364
